<a href="https://colab.research.google.com/github/Nikhileswar-Komati/Deep-Learning/blob/master/Weed_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
#kaggle username from api token 
os.environ['KAGGLE_USERNAME'] = "sindhumanchikanti"
#kaggle key from api token
os.environ['KAGGLE_KEY'] ="6f09a24a1b5600a06448e7e1341eb1bd"
#dataset path for downloading the data
!kaggle datasets download -d fpeccia/weed-detection-in-soybean-crops

100% 2.36G/2.37G [00:51<00:00, 39.3MB/s]
100% 2.37G/2.37G [00:51<00:00, 49.2MB/s]


In [2]:
#upload data
 
#ZipFile is a class of zipfile module for reading and writing zip files
from zipfile import ZipFile
#filename = zip file path 
file_name = "/content/weed-detection-in-soybean-crops.zip"
#to extract the zip file
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('done')

done


In [0]:
# !rm -rf test_data

In [0]:
# !rm -rf train_data

In [0]:
os.mkdir("/content/train_data")
os.mkdir("/content/train_data/broadleaf")
os.mkdir("/content/train_data/grass")
os.mkdir("/content/train_data/soil")
os.mkdir("/content/train_data/soybean")


In [0]:
os.mkdir("/content/test_data")
os.mkdir("/content/test_data/broadleaf")
os.mkdir("/content/test_data/grass")
os.mkdir("/content/test_data/soil")
os.mkdir("/content/test_data/soybean")

In [0]:
from shutil import copyfile 
import random

In [8]:
def split_data(source,training,testing,split_size):
  files = []
  for filename in os.listdir(source):
    file = source+filename
    files.append(filename)
    training_length = int(len(files)*split_size)
    shuffled_set = random.sample(files,len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[training_length:]
  for filename in training_set:
    this_file = source+filename 
    destination = training+filename 
    copyfile(this_file,destination)        
  for filename in testing_set:
    this_file = source+filename 
    destination = testing+filename 
    copyfile(this_file,destination)
#split_size = 0.8
split_data("/content/dataset/broadleaf/","/content/train_data/broadleaf/","/content/test_data/broadleaf/",0.8)
split_data("/content/dataset/grass/","/content/train_data/grass/","/content/test_data/grass/",0.8)
split_data("/content/dataset/soil/","/content/train_data/soil/","/content/test_data/soil/",0.8)
split_data("/content/dataset/soybean/","/content/train_data/soybean/","/content/test_data/soybean/",0.8)
print("done")

done


In [9]:
path, dirs, files = next(os.walk("/content/train_data/grass"))
print(len(files))

2816


In [10]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

Using TensorFlow backend.


In [11]:
train_dir = '/content/train_data'
test_dir = '/content/test_data'

train_gen_obj = ImageDataGenerator(rescale = 1.0/255.0)
test_gen_obj = ImageDataGenerator(rescale = 1.0/255.0)

train_gen = train_gen_obj.flow_from_directory(train_dir, batch_size = 64, target_size = (150, 150), class_mode = 'categorical')
test_gen = test_gen_obj.flow_from_directory(test_dir, batch_size = 32, target_size = (150, 150), class_mode = 'categorical')

Found 12267 images belonging to 4 classes.
Found 3069 images belonging to 4 classes.


In [0]:
import tensorflow as tf
from tensorflow import keras

In [0]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dense(4, activation='softmax')  
])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [0]:
# Compile Model. 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [19]:
# Train the Model
history = model.fit(train_gen, validation_data = test_gen, epochs = 5, verbose = 1)

Epoch 1/5
192/192 [==============================] - 60s 311ms/step - loss: 0.5009 - acc: 0.8029 - val_loss: 0.3054 - val_acc: 0.8791
Epoch 2/5
192/192 [==============================] - 59s 309ms/step - loss: 0.2583 - acc: 0.9001 - val_loss: 0.3012 - val_acc: 0.8788
Epoch 3/5
192/192 [==============================] - 59s 309ms/step - loss: 0.1751 - acc: 0.9373 - val_loss: 0.2066 - val_acc: 0.9218
Epoch 4/5
192/192 [==============================] - 59s 309ms/step - loss: 0.1115 - acc: 0.9580 - val_loss: 0.2044 - val_acc: 0.9228
Epoch 5/5
192/192 [==============================] - 59s 309ms/step - loss: 0.0770 - acc: 0.9733 - val_loss: 0.1869 - val_acc: 0.9299
